## <font color=orange> **What is an Agent:** <font/>
Agent is an AI model that task autonomous actions. it has tools and power of taking autonomous actions

## **Basic configuration**
The most common properties of an agent you'll configure are:

* ```name:``` A required string that identifies your agent.

* ```instructions:``` also known as a developer message or system prompt.

* ```model:``` which LLM to use, and optional model_settings to configure model tuning parameters like temperature, top_p, etc.

* ```tools:``` Tools that the agent can use to achieve its tasks.



In [ ]:
from agents import Agent, ModelSettings, function_tool

@function_tool
def get_weather(city: str) -> str:
     """returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="o3-mini",
    tools=[get_weather],
)

### **External Client:**

When use 3rd party LLM with Openai Agents SDK

```
external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
```

### **Model:**

Adding the model that we want to use with external client if 3rd party LLM
```
external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
```


### **Run Config:**

When running the agent we have to add few run time configurations

```
external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
```

### **Run Level Config:**
This configuration reflects the current operational state, when call the agent

### **Agent Level Config:**

Adding Configurations to indvisual or group of agents within a system.

### **Global Level Config:**

Adding configurations settingd to global level within a system or application.



## <font color=orange> **Runner Methods:** <font/>

### **Runner.run()**
This is an asynchronous method, for non blocking execution.
* Runs in async mode (you need await)
* Good for apps that already run in async environments
* Handles agent looping until done
* Can return any type of final output the agent is designed for

```
result = await Runner.run(
    starting_agent=my_agent,
    input="Hello!",
    max_turns=5
)
```

### **Runner.run_sync()**
This is  synchronous method, it warps Runner.run() internally and provides a blocking interface.
```
result = Runner.run_sync(
    starting_agent=my_agent,
    input="Hello!"
)
```
* Synchronous (no await)
* Perfect for small scripts
* Literally just wraps around run() but  blocks until finished
* Will not work if you already have an async event loop (like in Jupyter)

### **Runner.run_streamed()**
This is an asynchronous method that provides streaming capabilities, allowing for real-time updates.
```
stream = Runner.run_streamed(
    starting_agent=my_agent,
    input="Hello!"
)
for event in stream.stream_events():
    print(event)
```
* Starts the agent in streaming mode
* Lets you see events as they happen
* Useful for real-time UI updates or progress indicators
* Returns a RunResultStreaming object

**How the Loop Works**
Here’s the brain flow inside the runner:

1. Send input to agent
2. If final answer found → stop & return it
3. If agent hands off → run new agent
4. If agent wants tools → run them & repeat
5. If turns > max_turns → throw MaxTurnsExceeded
6. If guardrail trips → throw GuardrailTripwireTriggered



# <font color=orange> **Tools**<font/>
Think of Tools as superpowers you give your agent.

Without tools, an agent can only **think** and **respond** in text.

With tools, the agent can **do things** like call APIs, run calculations, look up a database, send emails, generate images, etc.

**Types of Tools**
###1. **Hosted Tools:**
These are ready-made tools running on OpenAI's servers, accessable via the [OpenAIResponsesModel]

You don’t have to write the logic or host anything yourself.

**Example:**

* **WebSearchTool:** Enables agents to perform web seraches.
* **FileSearchTool:** Allows retrieval of information from OpenAI Vecotor  Stores.
* **ComputerTool:** Facilitates automation of computer-based tasks.

```
from agents import Agent, FileSearchTool, Runner, WebSearchTool

agent = Agent(
    name="Assistant",
    tools=[
        WebSearchTool(),
        FileSearchTool(
            max_num_results=3,
            vector_store_ids=["VECTOR_STORE_ID"],
        ),
    ],
)

async def main():
    result = await Runner.run(agent, "Which coffee shop should I go to, taking into account my preferences and the weather today in SF?")
    print(result.final_output)

```

###2. **Function Calling:**
 This feature allows agents to utilize any python function as a tool, enhancing thier versatility.

 ```
from agents import Agent, FunctionTool, RunContextWrapper, function_tool

@function_tool  
async def fetch_weather(location: Location) -> str:
    
    """Fetch the weather for a given location.
    """
    # In real life, we'd fetch the weather from a weather API
    return "sunny"

agent = Agent(
    name="Assistant",
    tools=[fetch_weather],  
)

```

###3. **Agents as Tools:**
 Agents can employ other agents as tools, enbling hierarchical task management  without trnaferring  control.

```
from agents import Agent, Runner
import asyncio

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You translate the user's message to Spanish",
)

french_agent = Agent(
    name="French agent",
    instructions="You translate the user's message to French",
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools."
    ),
    tools=[
        spanish_agent.as_tool(
            tool_name="translate_to_spanish",
            tool_description="Translate the user's message to Spanish",
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French",
        ),
    ],
)

async def main():
    result = await Runner.run(orchestrator_agent, input="Say 'Hello, how are you?' in Spanish.")
    print(result.final_output)
```

In [ ]:
from agents.tool import function_tool

In [ ]:
@function_tool("get_weather")
def get_weather(location: str, unit: str = "C") ->str:
  """
  Fetch the weather for a given location, returning a short description.
  """
  #Example logic
  return f"The weather in {location} is 22 degree {unit}."

In [ ]:
@funtion_tool("giaic_student_finder")
def student_finder(student_roll: int) -> str:
  """
  find the GIAIC student based on their roll number
  """
  data = {
      1:"Ayesha",
      2:"Anum",
      3:"Fatima",
  }
  return data.get(student_roll, "Not Found")

In [ ]:
import asyncio

from agents import Agent, Runner

async def main():
  agent = Agent(
      name= "Assistant",
      instructions="You only respond in haikus.",
      tools=[get_weather, student_finder], # add tools here
      model=model
  )

  reult = await Runner.run(agent, "Share GIAIC roll no1 student details.")
  if __name__ == "__main__":
    asyncio.run(main)